# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import datetime
from scipy import stats #The SciPy stats module

In [3]:
import api_token  # handle iexcloud API tokens

env = "sandbox"  # "cloud"
BASE_URL, IEX_CLOUD_API_TOKEN = api_token.get_base_url(env=env), api_token.get_api_token(env=env)

In [4]:
TOP_N = 100

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [5]:
stock_file = 'sp_500_stocks.csv.gz'
stocks = pd.read_csv(stock_file, compression="gzip")

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [6]:
symbol = 'AAPL'
api_url =  f'{BASE_URL}/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'LCNAQAGLNR TK/( OAGDEMASL)TEESBS ',
 'calculationPrice': 'close',
 'open': 126.44,
 'openTime': 1676217905554,
 'openSource': 'iaiffocl',
 'close': 126.52,
 'closeTime': 1648401987257,
 'closeSource': 'fiaocilf',
 'high': 128.72,
 'highTime': 1655616309833,
 'highSource': '1 eilrntd5 cuypmdieeae ',
 'low': 123.5,
 'lowTime': 1667128143647,
 'lowSource': 'edmuyeinreia cel pd t15',
 'latestPrice': 125.9,
 'latestSource': 'Close',
 'latestTime': 'December 11, 2020',
 'latestUpdate': 1657176222485,
 'latestVolume': 89376575,
 'iexRealtimePrice': 123.97,
 'iexRealtimeSize': 103,
 'iexLastUpdated': 1612426259465,
 'delayedPrice': 127.86,
 'delayedPriceTime': 1686747243269,
 'oddLotDelayedPrice': 126.515,
 'oddLotDelayedPriceTime': 1615167236500,
 'extendedPrice': 123.01,
 'extendedChange': -0.14,
 'extendedChangePercent': -0.00119,
 'extendedPriceTime': 1663643692099,
 'previousClose': 123.3,
 'previousVolume': 83168540,
 '

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [7]:
pe_ratio = data['peRatio']
pe_ratio

38.98

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url =  f'{BASE_URL}/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,119.33,54.14,N/A
1,AAL,17.45,-1.24,N/A
2,AAP,159.17,23.63,N/A
3,AAPL,124.91,39.01,N/A
4,ABBV,109.42,23.57,N/A
...,...,...,...,...
500,YUM,110.65,32.86,N/A
501,ZBH,144.62,-197.69,N/A
502,ZBRA,382.88,44.97,N/A
503,ZION,41.84,18.57,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [10]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]   # filter out stock w/o positive earnings
final_dataframe = final_dataframe[:(TOP_N+1)]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [12]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [13]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,34.28,2,288
1,NLOK,19.89,3.93,497
2,UNM,24.16,5.01,409
3,BIO,592.08,5.09,16
4,KIM,15.37,6.6,644
...,...,...,...,...
96,BWA,38.17,17.58,259
97,HSIC,70.87,17.6,139
98,NTAP,63.58,17.8,155
99,HOLX,77.69,17.94,127


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio (PE)
* Price-to-book ratio (PB)
* Price-to-sales ratio (PS)
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [14]:
symbol = 'AAPL'
batch_api_call_url =  f'{BASE_URL}/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [15]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url =  f'{BASE_URL}/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,210.120,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,231.950,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.82,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.740,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.230,N/A,14.98,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,56.590,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,28.730,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,135.610,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.870,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1840.340,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, __we will replace missing data with the average non-`NaN` data point from that column.__

Here is the code to do this:

In [17]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [19]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# # Print each percentile score to make sure it was calculated properly
# for metric in metrics.values():
#     print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.93,N/A,54.00,0.861386,7.4000,0.724752,7.2200,0.827723,33.602115,0.853465,12.893535,0.805941,N/A
1,AAL,17.46,N/A,-1.26,0.141584,-1.8900,0.0455446,0.4226,0.0316832,-6.416008,0.0277228,1.420921,0.0574257,N/A
2,AAP,161.72,N/A,23.92,0.459406,2.8900,0.410891,1.1200,0.178218,11.106577,0.277228,2.469889,0.128713,N/A
3,AAPL,123.23,N/A,38.48,0.720792,32.5100,0.960396,7.6400,0.841584,26.788649,0.79604,20.093791,0.944554,N/A
4,ABBV,106.87,N/A,23.67,0.455446,12.3900,0.853465,4.7500,0.690099,16.200979,0.50099,9.384030,0.649505,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.01,N/A,32.12,0.647525,-4.0600,0.0415842,5.8800,0.760396,24.744230,0.768317,10.294266,0.691089,N/A
501,ZBH,148.47,N/A,-192.57,0.0217822,2.5900,0.368317,4.2800,0.617822,25.462927,0.780198,7.375208,0.481188,N/A
502,ZBRA,392.46,N/A,43.75,0.776238,10.7900,0.835644,4.7100,0.680198,29.838587,0.833663,10.936734,0.712871,N/A
503,ZION,41.94,N/A,18.02,0.350495,0.9684,0.0970297,2.3100,0.392079,10.404231,0.235644,2.705220,0.138614,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for __Robust Value__), which is the value score that we'll use to filter for 
stocks in this investing strategy.

__The higher RV Score, the more expensive the stock is__

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [20]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.93,N/A,54.00,0.861386,7.4000,0.724752,7.2200,0.827723,33.602115,0.853465,12.893535,0.805941,0.814653
1,AAL,17.46,N/A,-1.26,0.141584,-1.8900,0.0455446,0.4226,0.0316832,-6.416008,0.0277228,1.420921,0.0574257,0.0607921
2,AAP,161.72,N/A,23.92,0.459406,2.8900,0.410891,1.1200,0.178218,11.106577,0.277228,2.469889,0.128713,0.290891
3,AAPL,123.23,N/A,38.48,0.720792,32.5100,0.960396,7.6400,0.841584,26.788649,0.79604,20.093791,0.944554,0.852673
4,ABBV,106.87,N/A,23.67,0.455446,12.3900,0.853465,4.7500,0.690099,16.200979,0.50099,9.384030,0.649505,0.629901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.01,N/A,32.12,0.647525,-4.0600,0.0415842,5.8800,0.760396,24.744230,0.768317,10.294266,0.691089,0.581782
501,ZBH,148.47,N/A,-192.57,0.0217822,2.5900,0.368317,4.2800,0.617822,25.462927,0.780198,7.375208,0.481188,0.453861
502,ZBRA,392.46,N/A,43.75,0.776238,10.7900,0.835644,4.7100,0.680198,29.838587,0.833663,10.936734,0.712871,0.767723
503,ZION,41.94,N/A,18.02,0.350495,0.9684,0.0970297,2.3100,0.392079,10.404231,0.235644,2.705220,0.138614,0.242772


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [21]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True, ascending = True)   # the lower RV score, the more potential value
rv_dataframe = rv_dataframe[:(TOP_N+1)]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [22]:
portfolio_input()

Enter the value of your portfolio:1000000


In [23]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\19197\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.34,424,5.05,0.156436,0.4359,0.0475248,0.4034,0.0277228,2.569131,0.0336634,0.386290,0.0039604,0.0538614
1,AIG,39.64,249,-6.92,0.10495,0.5445,0.0554455,0.7363,0.0653465,4.537949,0.0415842,0.736772,0.0158416,0.0566337
2,AAL,17.46,567,-1.26,0.141584,-1.8900,0.0455446,0.4226,0.0316832,-6.416008,0.0277228,1.420921,0.0574257,0.0607921
3,MET,47.47,208,7.97,0.164356,0.5985,0.0594059,0.6735,0.0574257,4.313597,0.039604,0.623962,0.0118812,0.0665347
4,HFC,27.78,356,-29.04,0.0673267,0.8325,0.0792079,0.3190,0.0158416,6.436504,0.0673267,2.275655,0.118812,0.069703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,CXO,64.49,153,-1.26,0.141584,1.6500,0.233663,3.6100,0.566337,-10.462183,0.0237624,4.477185,0.265347,0.246139
97,BBY,104.30,94,15.64,0.29505,6.6900,0.689109,0.5930,0.0514851,6.695115,0.0811881,2.121473,0.114851,0.246337
98,NI,23.40,423,-29.59,0.0653465,1.9700,0.282178,1.9000,0.315842,10.871558,0.255446,5.279126,0.324752,0.248713
99,AZO,1175.80,8,16.27,0.313861,-30.8000,0.0118812,2.2100,0.373267,11.216888,0.283168,4.489403,0.269307,0.250297


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [29]:
today = datetime.datetime.now()
xlsx_file = f"003-value_strategy_{today.strftime('%Y-%m-%d')}.xlsx"

In [30]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [31]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [32]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [33]:
writer.save()